# Project 2. P2P  대출심사 알고리즘
**과목명:  응용데이터애널리틱스 (Applied Data Analytics)** \\
**학수번호:  INE4108** \\
**교강사:  송재욱 교수 (산업공학과)** \\
 \\
***

## 1. Environmental Set-up & Data Loading

In [1]:
import warnings

warnings.filterwarnings(action='ignore')

In [5]:
'''
from google.colab import drive


drive.mount('/content/drive', force_remount=True)

# enter the foldername in your Google Drive where you have saved the unzipped
FOLDERNAME =  'ADX/'

assert FOLDERNAME is not None, 'ERROR'

%cd drive/My\ Drive
%cp -r $FOLDERNAME ../../
'''
import pandas as pd
# 모델 수립을 위한 Train/Validation Set
df = pd.read_csv('P2_dataset.csv', encoding ='cp949') 

# 시스템 구현을 위한 Test Set 샘플
df_test = pd.read_csv('P2_dataset_test_sample.csv', encoding ='cp949')

In [ ]:
# import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# 한글 나눔포트 사용
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

from matplotlib import font_manager, rc
import seaborn as sns

plt.rc('font', family='NanumBarunGothic')

def plotHistogram(_df, columns):
    if type(columns) != list :
        print("columns 타입이 리스트가 아닙니다.")
        return
    for _n, i in enumerate(columns):
        df.describe()
        fig=plt.figure(figsize=(20,8))
        fig.patch.set_facecolor('xkcd:white')

        plt.subplot(2,3,_n + 1)
        plt.title(i)
        df.loc[:,i].hist(bins=50)
    plt.show()

Password:

## 2. Modelling
- df를 사용하여 예측모형 수립
- Feature engineering에 대한 토의/구현 진행
- Computation time을 고려하여 전체 데이터를 완전히 사용하지 말고 Sampling하여 Model Train/Vadliation을 진행하는 것을 추천
- Sampling을 단 한번 한 것으로 모형 Train 한 것이 과연 옳은 것인지에 대해 팀원들과 고민해볼 것. 해결 방법은 없을지 서치해보는 것도 하나의 task임.

In [ ]:
df.head()

In [ ]:
df['loan_status'].value_counts()

In [ ]:
df.shape

In [ ]:
for i in df.columns :
    X_i = df.loc[:,i]
    if X_i.dropna().shape[0]/df.shape[0] < 1:
      print("column {} : {} not null ratio".format(i,X_i.dropna().shape[0]/df.shape[0]))
      print("")
      print()
    if X_i.dtypes == 'object':
      print('column {} : '.format(i))
      print(df.loc[:,i].unique())
      print("")
      print()

In [4]:
df1=df.dropna(axis=0)
df1 = df1.replace({'sub_grade' :'A1'},1)
df1 = df1.replace({'sub_grade' : 'A2'},2)
df1 = df1.replace({'sub_grade' : 'A3'},3)
df1 = df1.replace({'sub_grade' : 'A4'},4)
df1 = df1.replace({'sub_grade' : 'A5'},5)
df1 = df1.replace({'sub_grade' : 'B1'},6)
df1 = df1.replace({'sub_grade' : 'B2'},7)
df1 = df1.replace({'sub_grade' : 'B3'},8)
df1 = df1.replace({'sub_grade' : 'B4'},9)
df1 = df1.replace({'sub_grade' : 'B5'},10)
df1 = df1.replace({'sub_grade' : 'C1'},11)
df1 = df1.replace({'sub_grade' : 'C2'},12)
df1 = df1.replace({'sub_grade' : 'C3'},13)
df1 = df1.replace({'sub_grade' : 'C4'},14)
df1 = df1.replace({'sub_grade' : 'C5'},15)
df1 = df1.replace({'sub_grade' : 'D1'},16)
df1 = df1.replace({'sub_grade' :'D2'},17)
df1 = df1.replace({'sub_grade' : 'D3'},18)
df1 = df1.replace({'sub_grade' : 'D4'},19)
df1 = df1.replace({'sub_grade' : 'D5'},20)
df1 = df1.replace({'sub_grade' : 'E1'},21)
df1 = df1.replace({'sub_grade' : 'E2'},22)
df1 = df1.replace({'sub_grade' : 'E3'},23)
df1 = df1.replace({'sub_grade' : 'E4'},24)
df1 = df1.replace({'sub_grade' : 'E5'},25)
df1 = df1.replace({'sub_grade' : 'F1'},26)
df1 = df1.replace({'sub_grade' : 'F2'},27)
df1 = df1.replace({'sub_grade' : 'F3'},28)
df1 = df1.replace({'sub_grade' : 'F4'},29)
df1 = df1.replace({'sub_grade' : 'F5'},30)
df1 = df1.replace({'sub_grade' : 'G1'},31)
df1 = df1.replace({'sub_grade' : 'G2'},32)
df1 = df1.replace({'sub_grade' : 'G3'},33)
df1 = df1.replace({'sub_grade' : 'G4'},34)
df1 = df1.replace({'sub_grade' : 'G5'},35)

df1=df1.drop('grade', axis=1)

df1 = df1.replace({'emp_length' :'10+ years'},10)
df1 = df1.replace({'emp_length' :'9 years'},9)
df1 = df1.replace({'emp_length' :'8 years'},8)
df1 = df1.replace({'emp_length' :'7 years'},7)
df1 = df1.replace({'emp_length' :'6 years'},6)
df1 = df1.replace({'emp_length' :'5 years'},5)
df1 = df1.replace({'emp_length' :'4 years'},4)
df1 = df1.replace({'emp_length' :'3 years'},3)
df1 = df1.replace({'emp_length' :'2 years'},2)
df1 = df1.replace({'emp_length' :'1 year'},1)
df1 = df1.replace({'emp_length' :'< 1 year'},0)

In [5]:
# 갚지 않은 사람의 비율이 낮은 원소부터 0, 1, ... 오름차순으로 order 부여

to_dummy = ['home_ownership', 'verification_status', 'purpose']
ratio_dicts = {}

for col in to_dummy:
  print('Column name : {}'.format(col))
  elements = df1[col].unique()
  ratios = {}
  for ele in elements:
    print('Element name : {}'.format(ele))
    temp_df = df1[df1[col] == ele]
    failed_people = temp_df[temp_df['loan_status'] == 'Charged Off']
    ratio = failed_people.shape[0]/temp_df.shape[0]
    print('Failed ratio : {}'.format(ratio))
    ratios[ele] = ratio
  print('')
  print()
  sorted_ratios = sorted(ratios.items()) # ratio 오름차순 정렬된 list
  ratio_dicts[col] = sorted_ratios

Column name : home_ownership
Element name : RENT
Failed ratio : 0.14945909043167735
Element name : MORTGAGE
Failed ratio : 0.10566304629454563
Element name : OWN
Failed ratio : 0.13404825737265416
Element name : OTHER
Failed ratio : 0.1796875
Element name : NONE
Failed ratio : 0.15151515151515152
Element name : ANY
Failed ratio : 0.0


Column name : verification_status
Element name : Source Verified
Failed ratio : 0.1337365818493112
Element name : Verified
Failed ratio : 0.13378387618808835
Element name : Not Verified
Failed ratio : 0.11866071923750367


Column name : purpose
Element name : debt_consolidation
Failed ratio : 0.13298969671259464
Element name : other
Failed ratio : 0.15249248880985958
Element name : credit_card
Failed ratio : 0.10667655578723464
Element name : house
Failed ratio : 0.15087719298245614
Element name : major_purchase
Failed ratio : 0.10560093348891482
Element name : home_improvement
Failed ratio : 0.10901109620777955
Element name : small_business
Failed ratio

In [6]:
df2 = df1.copy()

for col in to_dummy:
  ratios = ratio_dicts[col]
  for index, ele in enumerate(ratios):
    df2 = df2.replace(ele[0], index)

In [ ]:
df2.loc[:, to_dummy].head()

In [ ]:
'''
to_dummy = ['home_ownership', 'verification_status', 'purpose']
df2 = df1.drop(columns=to_dummy)
    
dummy_columns = []
for i in to_dummy:
  dummy = pd.get_dummies(df1[i])
  dummy_columns.append(dummy.columns)
  df2 = pd.concat([df2, dummy], axis=1, join='outer')
'''

In [7]:
df2 = df2.replace({'loan_status' :'Charged Off'},0)
df2 = df2.replace({'loan_status' :'Fully Paid'},1)
df2=df2.replace({'initial_list_status':'w'},0)
df2=df2.replace({'initial_list_status':'f'},1)

In [8]:
# Test Data 분할

def splitData(df_, ratio_, y_column):
    
    columns_ = df_.columns
    # Subsample the data
    mask = list(range(0, df_.shape[0], ratio_))
    X_val = df_.iloc[mask, :].drop(y_column, 1)
    y_val = df_.iloc[mask][y_column]
    
    mask = ~df_.index.isin(mask)
    X_train = df_.loc[mask, :].drop(y_column, 1)
    y_train = df_.loc[mask, y_column]
    
    return X_train, y_train, X_val, y_val

# train set, validation set split 
# train set, validation set = 7 : 1

X_train, y_train, X_val, y_val = splitData(df2, 8, y_column='loan_status')

'''
print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)
'''

'\nprint(X_train.shape)\nprint(X_val.shape)\nprint(y_train.shape)\nprint(y_val.shape)\n'

In [ ]:
# Test Data 분할
'''
from sklearn.model_selection import train_test_split

X_ = df2.drop(['loan_status', 'ANY', 'educational'], axis=1)
y_ = df2['loan_status']

X_train, y_train, X_val, y_val = train_test_split(X_, y_, test_size=0.125)

print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)
'''

In [ ]:
from sklearn.model_selection import train_test_split

dfs = df2.sample(n=3000)
print(dfs['loan_status'].value_counts())

X_ = dfs.drop(['loan_status'], axis=1)
y_ = dfs['loan_status']

X_train, X_val, y_train, y_val = train_test_split(X_, y_, test_size=0.2)

print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)

In [9]:
import numpy as np
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers

def Kernel_(x, y, params = 0, type_ = 'default') :
    if type_ == 'rbf' :
      Kernel = np.exp(- (np.sum(x ** 2, axis = 1).reshape(-1,1) + np.sum(y ** 2, axis = 1).reshape(1, -1) - 2 * x @ y.T) * params)
      return Kernel
      
    elif type_ == 'default' :
      Kernel = np.dot(x, y.T)
      return Kernel

def Minmax_(X) :
    return (X - X.min(axis = 0)) / (X.max(axis= 0) - X.min(axis = 0)), X.max(axis =0) , X.min(axis =0)

def Standar_(X) :
    return (X - X.mean(axis =0)) / X.std(axis = 0), X.mean(axis =0), X.std(axis = 0)

def Minmax_val(X, min, max) :
  return (X - min) / (max - min)

def Standar_val(X, avg, std) :
  return (X - avg) / std

def Convolution(pred, real) :
    pred = np.array(pred)
    y = np.array(real)
    TP = np.sum((pred == 1) & (y == 1))
    FP = np.sum((pred == 1) & (y != 1))
    FN = np.sum((pred != 1) & (y == 1))
    TN = np.sum((pred != 1) & (y != 1))
    return TP, FP, FN, TN

############## Accuracy is newly included!
def acc_precision_recall(X) :
  TP,FP,FN,TN = X
  return (TP + TN) / (TP + FP + FN + TN), TP / (TP + FP), TP / (TP + FN)

In [10]:
#Initializing values and computing H matrix. Note the 1. to force to float type
C = 10  # 여기 C는 hyperparameter이므로 변형 가능함 (이번에는 10으로 고정하고 진행)

X, cache_avg, cache_std = Standar_(np.array(X_train))
y = np.array(y_train)* 1.
X = np.nan_to_num(X)

y = y.reshape(-1,1) 
m, n = X.shape

In [ ]:
#Kernel 관련 정의 numpy array 형식으로 생성
H_lin = Kernel_(X, X) * 1.
H_lin *= y@y.T
P_lin = cvxopt_matrix(H_lin)
q = cvxopt_matrix(-np.ones((m, 1)))
G = cvxopt_matrix(np.vstack((-np.eye(m),np.eye(m))))
h = cvxopt_matrix(np.hstack((np.zeros(m), np.ones(m) * C)))
A = cvxopt_matrix(y.reshape(1, -1))
b = cvxopt_matrix(np.zeros(1))

#Run solver
sol = cvxopt_solvers.qp(P_lin, q, G, h, A, b)
alphas = np.array(sol['x'])

w = ((y * alphas).T @ X).reshape(-1,1)
S = ((alphas > 1e-4) & (alphas < C-1e-4)).flatten()
b = y[S] - np.sum(Kernel_(X, X[S], type_ = 'default')* y * alphas , axis = 0).reshape(-1,1)

print('Alphas = ',alphas[(alphas > 1e-4) & (alphas < C-1e-4)])
print('')
print('w = ', w.flatten())
print('')
print('b = ', np.mean(b))
print('')
print("support vector : ", np.array(range(m))[S])

In [ ]:
#####################################
#EFSVM 구현(Negative class(Yneg)들에 대한 Entropy
########################################

# You could use the following function to calculate the entropy
from scipy.stats import entropy

# You can use the following function to calculate the Euclidean distance (distance.cdist)
from scipy.spatial import distance

Ypos = y_train[y_train==1].index
Yneg = y_train[y_train!=1].index

Entropy_Yneg = pd.DataFrame()
distNeg = pd.DataFrame(distance.cdist(X_train, X_train.loc[Yneg], 'euclidean'), index=X_train.index, columns=Yneg)

In [ ]:
distNeg

,31933,35684,3246,14699,10308,26155,36766,37207,27236,17310,...,22357,26791,9542,13775,15837,20076,24981,15941,476,1726
105110,26525.620031,25825.760297,39341.620438,48356.113511,25428.258013,13113.490487,70542.165304,41768.223831,37028.615847,39316.455679,...,37166.573870,37528.362598,25416.723215,33213.402734,24922.734948,55299.650231,42260.878833,42135.623655,33126.585797,26593.795500
284664,28422.618802,24146.438582,56345.343749,59824.869206,16862.903764,21804.317197,72125.946202,64206.871794,46274.201654,44025.300741,...,40442.805429,16017.457423,15424.681238,16029.902201,15719.114839,51218.260990,22791.550044,40879.435267,37895.944991,24057.699134
205183,24029.159193,23735.098648,47502.226858,51800.746350,14570.880411,13603.434006,68704.490474,53149.077794,37392.460407,36840.087376,...,30624.715093,22901.694625,16947.523940,24978.833764,13486.287988,54657.498795,28241.820424,37910.003252,33767.737228,20623.196071
84018,34358.731293,30652.250014,57930.001697,62506.349906,11423.448726,13471.483858,81291.723877,62840.964557,51438.336391,47099.732841,...,45760.893670,18987.218234,12488.202693,26902.953466,17113.190025,61523.872830,22381.362106,49417.711747,44774.368200,31435.683331
276804,52008.314112,48652.470599,77128.849826,78747.111539,21106.194469,30988.290808,97808.666372,81779.438590,69234.427362,60311.634012,...,59616.162063,18352.249243,24547.146235,40826.825362,29635.005169,79130.890689,12843.430878,65242.483575,63783.384763,48342.649454
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56905,134171.462267,137160.265579,114692.656670,121545.290585,164361.570590,156414.029788,97669.557909,117373.282332,118782.343279,138765.993784,...,134178.378509,169794.123745,161601.520388,147669.771653,156913.861631,111617.044014,177320.545626,125050.865238,120730.045851,137357.012267
235263,20803.975926,19117.082721,45016.516921,49884.710281,16134.599076,13283.971765,66479.142624,51557.459181,35888.837027,35777.032996,...,31043.171400,24254.116535,16089.697097,20882.518247,13296.082047,50304.912862,30342.449406,35326.084757,30316.027618,17473.026252
247935,75823.104261,79913.053899,59311.253694,67521.825578,105916.405883,98807.974221,40913.252350,64414.338668,58418.688060,81992.252365,...,73922.077322,111089.553248,103923.008523,90831.590853,98473.207466,58131.535366,118494.685023,66726.843224,61872.162761,78597.339084
170529,30978.601693,32107.762912,36757.030572,47290.188924,32173.209297,19597.102890,71472.281812,36004.960449,36610.494212,40951.866462,...,38136.687956,44971.804182,33035.015111,41411.255333,31849.014606,59783.011636,49227.971650,45473.165999,35848.876592,32000.988667


In [ ]:
# Hyperparameter
C = 10
Gamma = 2 
k = 7 # Number of Nearest Neighbor
m = 10
beta = 1/18

In [ ]:
############################################
#EFSVM 구현 (Negative class (Yneg)들에 대한 Entropy)
#1. index 별로 Euclidean을 기준으로 가까운 k개 numP, numN, probP, probN, H 저장
########################################################

for i in Yneg:
    numP = np.sum(y_train.loc[distNeg.loc[:,i].sort_values()[1:k+1].index] == 1)
    numN = k - numP
    probP = numP/k
    probN = numN/k
    H = entropy([probP, probN])
    Entropy_Yneg.loc[i, 'numP'] = numP
    Entropy_Yneg.loc[i, 'numN'] = numN
    Entropy_Yneg.loc[i, 'probP'] = probP
    Entropy_Yneg.loc[i, 'probN'] = probN
    Entropy_Yneg.loc[i, 'H'] = H

In [ ]:
################################################
#EFSVM 구현 (subset 나누기, FM_l구하기, si 구하기)
#1. 구해진 엔트로피를 기준으로 구획을 m만큼 나누기
#2. 나뉘어진 subset을 FM_l을 할당 *1-beta*(m-l)
#3. Fuzzy membership s_i 설정 (Ypos =1, Yneg=FM_l)
################################################

Emax, Emin = Entropy_Yneg['H'].max(), Entropy_Yneg['H'].min()
FM = {}

for l in range(1, m+1):
    thrUp = Emin + l/m * (Emax - Emin)
    thrLow = Emin + (l-1)/m * (Emax - Emin)
    if m == l:
      Entropy_Yneg.loc[(Entropy_Yneg['H'] >= thrLow) &(Entropy_Yneg['H'] <= thrUp), 'subi' ] = l
      Entropy_Yneg.loc[(Entropy_Yneg['H'] >= thrLow) &(Entropy_Yneg['H'] <= thrUp), 'FM' ] = 1 - beta * (l-1)
    else:
      Entropy_Yneg.loc[(Entropy_Yneg['H'] >= thrLow) &(Entropy_Yneg['H'] < thrUp), 'subi' ] = l
      Entropy_Yneg.loc[(Entropy_Yneg['H'] >= thrLow) &(Entropy_Yneg['H'] < thrUp), 'FM' ] = 1 - beta * (l-1)

si = pd.DataFrame(index=X_train.index)
si.loc[Ypos, 'si']=1
si.loc[Entropy_Yneg.index, 'si'] = Entropy_Yneg['FM'].values
si = np.array(si['si'])

In [ ]:
xs, m = X.shape

#Kernel 관련 정의 numpy array 형식으로 생성
H = Kernel_(X,  X  , params = Gamma, type_ = 'rbf')* 1.
H *= y@y.T 
P = cvxopt_matrix(H)
q = cvxopt_matrix(-np.ones((xs, 1)))
G = cvxopt_matrix(np.vstack((-np.eye(xs),np.eye(xs))))
h = cvxopt_matrix(np.hstack((np.zeros(xs), si * C)))
A = cvxopt_matrix(y.reshape(1, -1))
b = cvxopt_matrix(np.zeros(1))

#Run solver
sol = cvxopt_solvers.qp(P, q, G, h, A, b)
alphas = np.array(sol['x'])

#Results
S = ((alphas > 1e-4) & (alphas < C-1e-4)).flatten()
b = y[S] - np.sum(Kernel_(X, X[S]  , params = 2, type_ = 'rbf')* y * alphas , axis = 0).reshape(-1,1)
print('Alphas = ',alphas[(alphas > 1e-4) & (alphas < C-1e-4)])
print('')
print('b = ', b[0])
print('')
print("support vector : ", np.array(range(xs))[S])

     pcost       dcost       gap    pres   dres
 0: -1.2378e+03 -3.7208e+04  7e+04  1e-01  5e-15
 1: -1.3474e+03 -3.9006e+03  4e+03  7e-03  1e-15
 2: -2.0909e+03 -2.4334e+03  5e+02  7e-04  2e-15
 3: -2.1944e+03 -2.1985e+03  5e+00  8e-06  2e-15
 4: -2.1955e+03 -2.1956e+03  5e-02  8e-08  2e-15
 5: -2.1956e+03 -2.1956e+03  5e-04  8e-10  1e-15
Optimal solution found.
Alphas =  [5.55555505 6.66666627 5.55555505 6.66666627 6.66666627 5.55555505
 5.55555505 5.55555505 5.55555505 4.99999943 6.66666627 5.55555505
 4.99999943 6.66666627 5.55555505 6.66666627 6.66666627 4.99999943
 5.55555505 5.55555505 5.55555505 4.99999943 5.55555505 6.66666627
 5.55555505 5.55555505 4.99999943 4.99999943 4.99999943 5.55555505
 4.99999943 5.55555505 6.66666627 5.55555505 5.55555505 6.66666627
 5.55555505 5.55555505 5.55555505 4.99999943 4.99999943 4.99999943
 4.99999943 4.99999943 6.66666627 6.66666627 6.66666627 4.99999943
 5.55555505 5.55555505 5.55555505 5.55555505 5.55555505 5.55555505
 4.99999943 5.5555550

In [ ]:
pred_sol = np.sign(np.sum(Kernel_(X, X, params = 2, type_ = 'rbf')* y * alphas , axis = 0).reshape(-1,1) + b[0])
acc_precision_recall(Convolution(pred_sol,y))

(0.4266666666666667, 0.8212435233160622, 0.44858490566037734)

## 3. System Implementation
- df_test에 기반하여 실제 사용을 위한 system implemetation 작업 진행
- Performance에 대한 평가데이터는 df_test로 지난 프로젝트와 마찬가지로 Data Pre-processing이 System implementation에 동시 구현
- 특히, scaler를 사용할 경우 cache를 반드시 사용하여 올바른 system implementation이 되도록 할 것!!! (주의!!!)

In [ ]:
df1_test=df_test.dropna(axis=0)

In [ ]:
df1_test = df1_test.replace({'sub_grade' :'A1'},1)
df1_test = df1_test.replace({'sub_grade' : 'A2'},2)
df1_test = df1_test.replace({'sub_grade' : 'A3'},3)
df1_test = df1_test.replace({'sub_grade' : 'A4'},4)
df1_test = df1_test.replace({'sub_grade' : 'A5'},5)
df1_test = df1_test.replace({'sub_grade' : 'B1'},6)
df1_test = df1_test.replace({'sub_grade' : 'B2'},7)
df1_test = df1_test.replace({'sub_grade' : 'B3'},8)
df1_test = df1_test.replace({'sub_grade' : 'B4'},9)
df1_test = df1_test.replace({'sub_grade' : 'B5'},10)
df1_test = df1_test.replace({'sub_grade' : 'C1'},11)
df1_test = df1_test.replace({'sub_grade' : 'C2'},12)
df1_test = df1_test.replace({'sub_grade' : 'C3'},13)
df1_test = df1_test.replace({'sub_grade' : 'C4'},14)
df1_test = df1_test.replace({'sub_grade' : 'C5'},15)
df1_test = df1_test.replace({'sub_grade' : 'D1'},16)
df1_test = df1_test.replace({'sub_grade' :'D2'},17)
df1_test = df1_test.replace({'sub_grade' : 'D3'},18)
df1_test = df1_test.replace({'sub_grade' : 'D4'},19)
df1_test = df1_test.replace({'sub_grade' : 'D5'},20)
df1_test = df1_test.replace({'sub_grade' : 'E1'},21)
df1_test = df1_test.replace({'sub_grade' : 'E2'},22)
df1_test = df1_test.replace({'sub_grade' : 'E3'},23)
df1_test = df1_test.replace({'sub_grade' : 'E4'},24)
df1_test = df1_test.replace({'sub_grade' : 'E5'},25)
df1_test = df1_test.replace({'sub_grade' : 'F1'},26)
df1_test = df1_test.replace({'sub_grade' : 'F2'},27)
df1_test = df1_test.replace({'sub_grade' : 'F3'},28)
df1_test = df1_test.replace({'sub_grade' : 'F4'},29)
df1_test = df1_test.replace({'sub_grade' : 'F5'},30)
df1_test = df1_test.replace({'sub_grade' : 'G1'},31)
df1_test = df1_test.replace({'sub_grade' : 'G2'},32)
df1_test = df1_test.replace({'sub_grade' : 'G3'},33)
df1_test = df1_test.replace({'sub_grade' : 'G4'},34)
df1_test = df1_test.replace({'sub_grade' : 'G5'},35)

df1_test = df1_test.drop('grade', axis=1)

df1_test = df1_test.replace({'emp_length' :'10+ years'},10)
df1_test = df1_test.replace({'emp_length' :'9 years'},9)
df1_test = df1_test.replace({'emp_length' :'8 years'},8)
df1_test = df1_test.replace({'emp_length' :'7 years'},7)
df1_test = df1_test.replace({'emp_length' :'6 years'},6)
df1_test = df1_test.replace({'emp_length' :'5 years'},5)
df1_test = df1_test.replace({'emp_length' :'4 years'},4)
df1_test = df1_test.replace({'emp_length' :'3 years'},3)
df1_test = df1_test.replace({'emp_length' :'2 years'},2)
df1_test = df1_test.replace({'emp_length' :'1 year'},1)
df1_test = df1_test.replace({'emp_length' :'< 1 year'},0)
df1_test

In [ ]:
df2_test = df1_test.copy()

for col in to_dummy:
  ratios = ratio_dicts[col]
  for index, ele in enumerate(ratios):
    df2_test = df2_test.replace(ele[0], index) # ele[0] : 원소 이름, index : ratio가 오름차순으로 정렬되어서, 0이 가장 ratio가 가장 낮음

In [ ]:
df2_test = df2_test.replace({'loan_status' :'Charged Off'},0)
df2_test = df2_test.replace({'loan_status' :'Fully Paid'},1)
df2_test = df2_test.replace({'initial_list_status':'w'},0)
df2_test = df2_test.replace({'initial_list_status':'f'},1)

In [ ]:
X_test = Standar_val(np.array(df2_test.drop(columns=['loan_status'])), avg=cache_avg, std=cache_std)
X_test = np.nan_to_num(X_test)

y_test = np.array(df2_test['loan_status'])* 1.
y_test = y_test.reshape(-1,1)

In [ ]:
# Solution
pred_sol = np.sign(np.sum(Kernel_(X, X  ,type_ = 'default')* y, axis = 0).reshape(-1,1) + b[0])
acc_precision_recall(Convolution(pred_sol,y))

(0.64, 0.990625, 0.5981132075471698)

In [ ]:
xs, m = X_test.shape

#Kernel 관련 정의 numpy array 형식으로 생성
H = Kernel_(X_test,  X_test, params = Gamma, type_ = 'rbf')* 1.
H *= y_test@y_test.T 
P = cvxopt_matrix(H)
q = cvxopt_matrix(-np.ones((xs, 1)))
G = cvxopt_matrix(np.vstack((-np.eye(xs),np.eye(xs))))
h = cvxopt_matrix(np.hstack((np.zeros(xs), si * C)))
A = cvxopt_matrix(y.reshape(1, -1))
b = cvxopt_matrix(np.zeros(1))

#Run solver
sol = cvxopt_solvers.qp(P, q, G, h, A, b)
alphas = np.array(sol['x'])

#Results
S = ((alphas > 1e-4) & (alphas < C-1e-4)).flatten()
b = y[S] - np.sum(Kernel_(X, X[S]  , params = 2, type_ = 'rbf')* y * alphas , axis = 0).reshape(-1,1)
print('Alphas = ',alphas[(alphas > 1e-4) & (alphas < C-1e-4)])
print('')
print('b = ', b[0])
print('')
print("support vector : ", np.array(range(xs))[S])

TypeError: ignored

In [ ]:
pred_sol = np.sign(np.sum(Kernel_(X_test, X_test, params = 2, type_ = 'rbf') * y_test , axis = 0).reshape(-1,1) + b[0])
acc_precision_recall(Convolution(pred_sol, y_test))

1. 전체적으로 다 같이 코드 리뷰하면서 변수 이름 정리
2. 마지막 예측 차원 안 맞는 문제 해결 (alphas)
3. 최적 Parameter 탐색 위해서 SVM fit하고 예측하는 과정 함수로 만들기
4. 변수 별로 Scaling 방법 결정하기 (Standard, Min-Max, Log)
5. Linear kernel SVM 할 때 b가 nan으로 나오는 문제 해결하기